# Power Method
Find the largest eigenvalue [See Here](https://www.youtube.com/watch?v=yBiQh1vsCLU)
* Let A be a Matrix and x0=[1,0,....0]

In [85]:
import numpy as np
def power_method(A,k=10):
    m,n=A.shape
    if m!=n: return 'The matrix should be squer'
    x=np.zeros(n)
    x[0]=1.0
    for i in range(k):
        p=np.dot(A,x)
        m=np.max(np.abs(p))
        x=p/m 
        # p/la.norm(p)
    return m

### Example 1 (Easy)

In [84]:
A=np.array([[7,9],[9,7]])
# From linear algebra we can find that A has two eigenvalue 16 and -2. 
power_method(A)

15.999999895691872

### Invers Problem for Eigenvalue
But does **power_method** work for a big dimention matrix?<br>
So lets to check it. Please give me a matrix of dim 50 that its largest eigenvalue is 23.<br>
Is it difficult?<br>
So we want do it by QR factorization.<br>

* **Goal:** Make a matrix such that its eigenvalue are the element of an arbitrary array.

In [71]:
from scipy import linalg as la
def eigenvalue_invers_problem(b):
    n=len(b)
    D=np.diag(b)
    R=np.random.rand(n,n)
    q,r = la.qr(R)
    B=np.dot(np.transpose(q),np.dot(D,q))
    return B

### Example 2

In [82]:
K=eigenvalue_invers_problem(np.array([12,-2,7]))
power_method(K)

12.035859204704591

### Example 3

In [86]:
n=7
b=np.linspace(1,n,n)
print(b)
B=eigenvalue_invers_problem(b)
power_method(B)

[1. 2. 3. 4. 5. 6. 7.]


6.675158013863618